In [1]:
import glob
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
import json
import stanza
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
from itertools import combinations
from collections import Counter
import pandas as pd
# import gender_guesser.detector as gender

c:\Users\desus\anaconda3\envs\ds\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\desus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\desus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [4]:
names_df = pd.read_csv('disney_names_and_genders.csv', header=0)
names_df['Name'] = names_df['Name'].str.lower()
names_df.head(3)

,Unnamed: 0,Name,Gender
0,0,.giffany,F
1,1,22,F
2,2,3rd street bandits,M


In [5]:
movies_path = "movies/*.txt"
files = glob.glob(movies_path)

In [6]:
nlp = stanza.Pipeline(lang="en", processors="tokenize,pos,lemma,ner")
stop_words = set(stopwords.words("english"))
# d = gender.Detector()

2025-11-16 21:33:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-11-16 21:33:50 INFO: Downloaded file to C:\Users\desus\stanza_resources\resources.json
2025-11-16 21:33:50 WARNING: Language en package default expects mwt, which has been added
2025-11-16 21:33:51 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| ner       | ontonotes-ww-multi_charlm |

2025-11-16 21:33:51 INFO: Using device: cpu
2025-11-16 21:33:51 INFO: Loading: tokenize
2025-11-16 21:33:56 INFO: Loading: mwt
2025-11-16 21:33:56 INFO: Loading: pos
2025-11-16 21:34:01 INFO: Loading: lemma
2025-11-16 21:34:02 INFO: 

In [7]:
def fix_gpe(tokens, entities):
    """
    tokens: list of words in a sentence
    entities: list/set of (entity_text, entity_type)
    Returns:
        - new token list with multi-word entities joined by '-'
        - updated set of entities including merged versions
    """
    tokens = [tok for tok in tokens if tok is not None]
    text_str = " ".join(tokens)
    merged_entities = set()

    for entity_text, ent_type in entities:
        if " " in entity_text:  # only merge multi-word entities
            merged_text = entity_text.replace(" ", "-")
            pattern = r'\b' + re.escape(entity_text) + r'\b'
            text_str = re.sub(pattern, merged_text, text_str)
            merged_entities.add((merged_text, ent_type))
        else:
            merged_entities.add((entity_text, ent_type))

    new_tokens = text_str.split(" ")
    return new_tokens, merged_entities


In [20]:
def get_gender(name):

    # check if name is not an empty string
    if name is None:
        return 'Unknown'

    try:
        name = name.lower().strip()
        if name == '':
            return 'Unknown'
    
        # if exact match is found
        gender = names_df.loc[names_df['Name'] == name]['Gender'].values[0]

    except:
        # else try to split name and find parts
        genders = []
        name_parts = name.lower().split(' ')
        for part in name_parts:
            if part != '':
                # if part of the name is found in any of the rows, get name and gender
                gender = names_df[names_df.Name.str.contains(part)]['Gender'].to_list()
                genders += gender
        # count gender occurrences 
        gender_counts = Counter(genders)
        if len(gender_counts.keys()) == 1:
            gender = list(gender_counts.keys())[0]
        else:
            if len(gender_counts.keys()) > 1:
                gender = sorted(gender_counts.items(), key=lambda x: x[1], reverse=True)[0][0]
            else:
                # length of unq genders is 0
                gender = 'Unknown'
    
    return gender


In [25]:
get_gender('mul')

'M'

In [19]:
pattern = r"PLOT:\s*(.*)"  
window_size = 2
pairs = set()
pos_tags = {}

for file_idx, file_path in enumerate(files):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    match = re.search(pattern, text, re.S)

    if match:
        plot = match.group(1).strip()
        doc = nlp(plot)
        # Collect all GPE entities with type
        # entities = set((ent.text, ent.type) for ent in doc.ents if ent.type =="GPE")
        person_or_gpe = []
        for ent in doc.ents:
            if ent.type == "PERSON":
                # Take only the first token of the PERSON entity
                first_word = re.split(r"['\-]", ent.text.split()[0])[0]
                person_or_gpe.append((first_word, ent.type))
            elif ent.type == "GPE":
                # For GPE, keep full entity
                person_or_gpe.append((ent.text, ent.type))
        entities = list(set(person_or_gpe))

        for sent in doc.sentences:
            sent = sent.to_dict()
            sent_tokens = []
            for word in sent:
                lemma = word.get('lemma')
                if lemma is not None:
                    lemma = lemma.lstrip('-')
                pos = word.get('upos')
                ner = word.get('ner')

                #detect gender of beginning of person names or single token person names
                if ner in {"B-PERSON", "S-PERSON"}:
                    print(lemma)
                    gender = get_gender(lemma)
                else:
                    gender = None

                # Filter tokens
                if pos not in {"PUNCT", "PART", "I-PERSON"} and lemma not in stop_words:
                    sent_tokens.append(lemma) 
                    pos_tags[lemma] = {"pos": pos, "gender": gender}

            sent_tokens, entities_merged = fix_gpe(sent_tokens, entities)
     
            for ent, pos in entities_merged:
                gender_ent = get_gender(ent) if pos == "PERSON" else None
                pos_tags[ent] = {"pos": pos, "gender": gender_ent}
            
            n = len(sent_tokens)
            for i in range(n):
                window = sent_tokens[i : i + window_size]
                for w1, w2 in combinations(window, 2):
                    pairs.add((w1, w2))

    print(f"file {file_idx} completed: {file_path}")

Roger
Roger
Pongo
Perdita
Roger
Pongo
Perdita
Anita
Roger
Anita
Perdita
Pongo
Anita
Cruella
Skinner
Anita
Cruella
Perdita
Anita
Cruella
Perdita
Roger
Anita
Cruella
Anita
Roger
Jasper
Horace
Pongo
Perdita
Cruella
Skinner
Pongo
Roger
Anita
Anita
Cruella
Roger
Kipper
Jasper
Horace
Pongo
Perdita
Cruella
Jasper
Horace
Cruella
Skinner
Cruella
Cruella
Pongo
Perdita
Roger
Anita
Nanny
Roger
Cruella
Roger
Anita
file 0 completed: movies\101 Dalmatians (1996 film).txt
Cruella
Pavlov
Cruella
Alonzo
Kevin
Chloe
Cruella
None


TypeError: object of type 'NoneType' has no len()

In [ ]:
# ---- Save pairs ----
with open("cooccurrence_pairs_window2.txt", "w", encoding="utf-8") as f:
    for w1, w2 in pairs:
        f.write(f"{w1}\t{w2}\n")

# ---- Save POS tags ----
with open("pos_tags.json", "w", encoding="utf-8") as f:
    json.dump(pos_tags, f, indent=2)